Notes based on lectures
and Michael Nielsens book at
http://neuralnetworksanddeeplearning.com/

# Universal approximation theorem
The great efficacy of neural networks can largely be contributed to the universal approximation theorem, which can simply be written as:

$$
\vert F(\boldsymbol{x})-f(\boldsymbol{x};\boldsymbol{\Theta})\vert < \epsilon \hspace{0.1cm} \forall \boldsymbol{x}\in[0,1]^d.
$$

Where $F(\boldsymbol{x})$ is a continuous and deterministic function defined on the unit cube in $d$-dimensions, $F : [0,1]^d \rightarrow \mathbb{R} $. We aim to approximate $F$ for any given small positive error tolerance $\epsilon > 0$. $f(\boldsymbol{x}; \boldsymbol{\Theta})$ is a one-layer hidden neural network with parameters $\boldsymbol{\Theta} = (\boldsymbol{W},\boldsymbol{b})$ where the weights $\boldsymbol{W}\in\mathbb{R}^{m\times n}$ and the biases $\boldsymbol{b}\in \mathbb{R}^{n}$. 

The neural network $f$ takes the input and computes for each neuron the activation value $\boldsymbol{z_i}$ as: 
$$\boldsymbol{z_i} = \boldsymbol{w_i}\cdot \boldsymbol{x} + b_i$$
Where $\boldsymbol{w_i}$ and $b_i$ are the weight vector and bias for each neuron $i$. This value is then for each neuron passed through a continuous sigmoidal function $\sigma(\boldsymbol{z})$. Otherwise known as an activation function. This function has the property: 
$$
\sigma(\boldsymbol{z}) = \left\{\begin{array}{cc} 1 & \boldsymbol{z}\rightarrow \infty\\ 0 & \boldsymbol{z} \rightarrow -\infty \end{array}\right.
$$
Such a function could be for example the standard logistic function: 
$$ \sigma(\boldsymbol{z}) = \frac{1}{1+\mathrm e^{-\boldsymbol{z}}} $$
The function introduces non-linearity, which enables the possibility of approximating functions of high complexity.

By summing the outputs of all the neurons, weighted directly by the network's parameters, we approximate $F(\boldsymbol{x})$. Specifically, if we denote each neuron's activated output as $\sigma(z_i)$, where $z_i = \boldsymbol{w_i} \cdot \boldsymbol{x} + b_i$, then the final output of the neural network can be expressed as:
$$
f(\boldsymbol{x}, \boldsymbol{\Theta}) = \sum_{i=1}^{m} \sigma(z_i)
$$
Each neuron's contribution is implicitly scaled by the weights $\boldsymbol{w_i}$ and biases $b_i$ in the network. Through careful adjustment of these parameters, we can control each neuron's impact on the overall approximation, allowing $f(\boldsymbol{x}; \boldsymbol{\Theta})$ to closely approximate $F(\boldsymbol{x})$ within the given error tolerance $\epsilon$.

In single-layer networks, the Universal Approximation Theorem guarantees that with sufficient neurons, the right weights $\boldsymbol{W}$, and biases $\boldsymbol{b}$, we can approximate any continuous function. For multi-layer networks, which often enhance approximation power, backpropagation enables efficient tuning of parameters across layers through gradient descent methods.


# Back propagation
Back propagation is a process in which we attempt to improve a neural network by adjusting the internal parameters through calculating gradients of the cost/loss function wrt these parameters. We examine the initial predictions of the network, and then work backwards from the output, adjusting each layer of neurons using the chain rule of calculus.

### Forward pass
First we do a forward pass, where we have an input that goes trough the network and predicts an output. Again we're interested in the activation values, but now we look at multiple layers, which is determined by the immeadiately preceding layer. The expression for each neuron $j$ in layer $l$ is then:

$$
z_j^l = \sum_{i=1}^{M_{l-1}}w_{ij}^la_i^{l-1}+b_j^l
$$

Here $a_i^{l-1}$ is the output from neuron $i$ in the previous layer $\sigma(z_i^{l-1})$. The total number of neurons is $M_{l-1}$.

### Error calculation
We then use a cost/loss function such as MSE or Cross-Entropy Loss to evaluate how far off the target our prediction is. Our goal is to minimize the function.

The MSE cost function is defined as:

$$
{\cal C}(\boldsymbol{\Theta})  =  \frac{1}{2}\sum_{i=1}^n\left(y_i - \tilde{y}_i\right)^2
$$

Since the output layer is our starting point we consider the cost function at $l = L$, which is the final layer:
$$
{\cal C}(\boldsymbol{\Theta}^L)  =  \frac{1}{2}\sum_{i=1}^n\left(y_i - \tilde{y}_i\right)^2=\frac{1}{2}\sum_{i=1}^n\left(y_i - a_i^L\right)^2
$$

Taking the derrivative wrt the weights, we get:
$$
\frac{\partial{\cal C}(\boldsymbol{\Theta}^L)}{\partial w_{jk}^L}  =  \left(y_j - a_j^L\right)\frac{\partial a_j^L}{\partial w_{jk}^{L}}
$$
Note that previously we considered $w_{ij}^{l}$, but in the output layer we consider $w_{jk}^{L}$, which is strictly just a change of notation and does not affect the calculation of the derrivatives. $j$ indexes neurons in the output layer $L$, and $k$ indexes neurons in the preceding layer $L-1$.

By applying the chain rule we can find an alternative way of calculating $\frac{\partial a_j^L}{\partial w_{jk}^{L}}$ as such:

$$\frac{\partial a_j^L}{\partial w_{jk}^{L}} =\frac{\partial a_j^L}{\partial z_{j}^{L}}\frac{\partial z_j^L}{\partial w_{jk}^{L}}$$

In this case we consider the sigmoid activation function $\sigma(z_j^l)$, which has the derrivative $\sigma(z_j^l)(1-\sigma(z_j^l))$. This leads to the following: 
$$\frac{\partial a_j^l}{\partial z_j^{l}} = a_j^l(1-a_j^l)$$

From the definition of $z_j^l$ we get $\frac{\partial z_j^l}{\partial w_{ij}^l} = a_i^{l-1}$. As we put it all together we get the final result:

$$
\frac{\partial{\cal C}(\boldsymbol{\Theta}^L)}{\partial w_{jk}^L}  =  \left(y_j - a_j^L\right)a_j^L(1-a_j^L)a_k^{L-1}
$$

To simplify both notation and calculations, we introduce a new term:
$$\delta_j^l \equiv \frac{\partial C}{\partial z_j^l}$$
This is known as the error in the $j^{th}$ neuron in the $l^{th}$ layer. We want to relate this quantity to $\frac{\partial C}{\partial w_{jk}^l}$ and $\frac{\partial C}{\partial b_j^l}$. We can again apply the chain rule for the output layer to obtain:
$$\delta_j^L = \frac{\partial C}{\partial a_j^L} \cdot \frac{\partial a_j^L}{\partial z_j^L}$$

The second term on the right have already been found, and the first term is simply:
$$\frac{\partial C}{\partial a_j^L} = a_j^L - y_j$$

Bringing it together:
$$\delta_j^L = (a_j^L - y_j) \cdot a_j^L(1-a_j^L)$$

And the expression for the weight gradient of the cost function can then be written as:
$$\frac{\partial C}{\partial w_{jk}^L} = \delta_j^L \cdot a_k^{L-1}$$

Now we want to be able to calculate the error for layer $l$ based on layer $l+1$. To do this we apply the chain rule once again and sum over all k as such:
$$\delta_j^l = \frac{\partial C}{\partial z_j^l} = \sum_k \frac{\partial {\cal C}}{\partial z_k^{l+1}}\frac{\partial z_k^{l+1}}{\partial z_j^{l}}=\sum_k \delta_k^{l+1}\frac{\partial z_k^{l+1}}{\partial z_j^{l}}$$

From the definition of $z_j^l$ we have:
$$
z_j^{l+1} = \sum_{i=1}^{M_{l}}w_{ij}^{l+1}\sigma(z_j^{l})+b_j^{l+1}
$$

Which brings us to an expression of $\delta_j^l$ in terms of the weights:
$$
\delta_j^l =\sum_k \delta_k^{l+1}w_{kj}^{l+1}\sigma'(z_j^l) = (\sum_k \delta_k^{l+1}w_{kj}^{l+1}) \cdot a_j^l(1-a_j^l)
$$

In terms of the bias we have:
$$\delta_j^l = \frac{\partial {\cal C}}{\partial b_j^l}\frac{\partial b_j^l}{\partial z_j^l} = \frac{\partial C}{\partial b_j^l}$$

### Back propagation with learning rate $\eta$
Now that we have expressions for the error, we can use gradient descent with learning rate $\eta$ to update the weights and biases:
$$
w_{jk}^l\leftarrow  = w_{jk}^l- \eta \delta_j^la_k^{l-1}
$$

$$
b_j^l \leftarrow b_j^l-\eta \frac{\partial {\cal C}}{\partial b_j^l}=b_j^l-\eta \delta_j^l
$$